In [1]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import FunctionTransformer

In [2]:
ILUMINA_PATH = Path('../data/Ilumina')
ILUMINA_CLEAN_PATH = Path('../data/Ilumina/clean')

NANOPORE_PATH = Path('../data/Nanopore')
NANOPORE_CLEAN_PATH = Path('../data/Nanopore/clean')

In [3]:
columns = [i for i in range(14) if i != 4] #31
SampleInfo = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Sample info + Sperm quality", skipfooter=1, skiprows=1, usecols=columns, keep_default_na=False, 
            na_values=["null", ""])

columns = [i for i in range(31) if i not in range(2,14)]
SpermQuality = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Sample info + Sperm quality", skipfooter=1, skiprows=1, usecols=columns, keep_default_na=False, 
            na_values=["null", ""])

IluminaPylum = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Pylum-level microbiota")
IluminaFamily = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Family-level microbiota")
IluminaGenus = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Genus-level microbiota")

NanoporePylum = pd.read_excel(NANOPORE_PATH / "Relative abundance of seminal microbiota from idiopathic infertile patients and donors using MinION sequencing.xlsx",
            sheet_name="Pylum-level microbiota")
NanoporeFamily = pd.read_excel(NANOPORE_PATH / "Relative abundance of seminal microbiota from idiopathic infertile patients and donors using MinION sequencing.xlsx",
            sheet_name="Family-level microbiota")
NanoporeGenus = pd.read_excel(NANOPORE_PATH / "Relative abundance of seminal microbiota from idiopathic infertile patients and donors using MinION sequencing.xlsx",
            sheet_name="Genus-level microbiota")

In [11]:
IluminaPylum = IluminaPylum.replace({0:-1})
IluminaFamily = IluminaFamily.replace({0:-1})
IluminaGenus = IluminaGenus.replace({0:-1})

NanoporePylum = NanoporePylum.replace({0:-1})
NanoporeFamily = NanoporeFamily.replace({0:-1})
NanoporeGenus = NanoporeGenus.replace({0:-1})

In [12]:
IluminaPylum["Fertil"] = 1
IluminaPylum.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

IluminaFamily["Fertil"] = 1
IluminaFamily.loc[(IluminaFamily.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

IluminaGenus["Fertil"] = 1
IluminaGenus.loc[(IluminaGenus.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

NanoporePylum["Fertil"] = 1
NanoporePylum.loc[(NanoporePylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

NanoporeFamily["Fertil"] = 1
NanoporeFamily.loc[(NanoporeFamily.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

NanoporeGenus["Fertil"] = 1
NanoporeGenus.loc[(NanoporeGenus.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

In [13]:
IluminaFamily.describe()

,Peptoniphilaceae,Campylobacteraceae,Streptococcaceae,Moraxellaceae,Staphylococcaceae,Prevotellaceae,Corynebacteriaceae,Peptostreptococcaceae,Lactobacillaceae,Veillonellaceae,...,Limnochordaceae,Chthonomonadaceae,Egicoccaceae,Jiangellaceae,Orbaceae,Bernardetiaceae,Succinivibrionaceae,Sporolactobacillaceae,Gemmataceae,Fertil
count,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,...,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000
mean,32.102036,5.741070,4.781684,4.085532,3.808041,3.680070,3.272388,2.787459,2.572998,2.537082,...,-0.928518,-0.928525,-0.928525,-0.928527,-0.964241,-0.946387,-0.964246,-0.946389,-0.964248,0.250000
std,20.070654,9.230076,6.950188,8.898362,6.791023,6.273797,4.501437,4.803067,9.336237,3.107507,...,0.260065,0.260039,0.260039,0.260033,0.187490,0.227382,0.187462,0.227374,0.187453,0.436931
min,0.011100,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.021200,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,16.110100,0.011850,0.056625,0.014925,0.183025,0.087500,0.193575,0.025350,0.034425,0.108375,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
50%,32.372600,0.868900,0.853650,0.287500,1.159250,0.898800,1.023000,0.571550,0.140850,0.919500,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
75%,48.391150,7.789350,5.876700,3.467100,4.204325,3.265300,5.268325,4.565900,0.674850,5.112225,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.250000
max,66.937300,36.337200,24.560700,42.279100,30.683200,21.057400,19.638600,21.208600,63.739100,12.531100,...,0.001400,0.000800,0.001000,0.000800,0.001800,0.001400,0.001800,0.001400,0.001800,1.000000


In [16]:
encode =  KBinsDiscretizer(11, encode='ordinal', strategy='uniform', subsample=None, random_state=2707)

transformer = FunctionTransformer(encode.fit_transform)

IluminaPylumCod = IluminaPylum.copy()
IluminaFamilyCod = IluminaFamily.copy()
IluminaGenusCod = IluminaGenus.copy()

IluminaPylumCod.iloc[:,range(1,IluminaPylumCod.shape[1]-1)] = transformer.transform(IluminaPylumCod.iloc[:,range(1,IluminaPylumCod.shape[1]-1)])
IluminaFamilyCod.iloc[:,range(1,IluminaFamilyCod.shape[1]-1)] = transformer.transform(IluminaFamilyCod.iloc[:,range(1,IluminaFamilyCod.shape[1]-1)])
IluminaGenusCod.iloc[:,range(1,IluminaGenusCod.shape[1]-1)] = transformer.transform(IluminaGenusCod.iloc[:,range(1,IluminaGenusCod.shape[1]-1)])

NanoporePylum.iloc[:,range(1,NanoporePylum.shape[1]-1)] = transformer.transform(NanoporePylum.iloc[:,range(1,NanoporePylum.shape[1]-1)])
NanoporeFamily.iloc[:,range(1,NanoporeFamily.shape[1]-1)] = transformer.transform(NanoporeFamily.iloc[:,range(1,NanoporeFamily.shape[1]-1)])
NanoporeGenus.iloc[:,range(1,NanoporeGenus.shape[1]-1)] = transformer.transform(NanoporeGenus.iloc[:,range(1,NanoporeGenus.shape[1]-1)])

In [17]:
IluminaPylumCod.describe()

,Firmicutes,Proteobacteria,Actinobacteria,Tenericutes,Bacteroidetes,Armatimonadetes,Spirochaetes,Planctomycetes,Verrucomicrobia,Chloroflexi,...,Dictyoglomi,Balneolaeota,Fibrobacteres,Chlamydiae,Acidobacteria,Kiritimatiellaeota,Nitrospirae,candidate.division.Zixibacteria,Thermodesulfobacteria,Fertil
count,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,...,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000
mean,6.375000,2.803571,2.053571,0.625000,1.750000,7.392857,6.267857,7.071429,3.035714,5.892857,...,3.928571,2.142857,1.785714,3.571429,1.946429,1.071429,1.785714,1.071429,1.964286,0.250000
std,2.512243,2.596639,2.109887,1.968502,3.016621,3.295609,3.048046,3.939807,4.639609,4.964157,...,4.928054,4.140393,3.864591,4.834938,3.272028,3.120939,3.864591,3.120939,4.008919,0.436931
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,1.000000,0.000000,0.000000,0.000000,8.000000,7.000000,9.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.000000,2.000000,1.000000,0.000000,0.000000,9.000000,7.000000,9.000000,0.000000,10.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.000000,4.250000,3.000000,0.000000,2.250000,9.000000,8.000000,9.000000,10.000000,10.000000,...,10.000000,0.000000,0.000000,10.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.250000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000


In [27]:
# encodeOH = KBinsDiscretizer(n_bins=20, encode='onehot-dense', strategy='uniform', subsample=None, random_state=2707)

IluminaPylumCodOH = IluminaPylumCod.copy()
IluminaFamilyCodOH = IluminaFamilyCod.copy()
IluminaGenusCodOH = IluminaGenusCod.copy()

#IluminaPylumCodOH
columns = IluminaPylumCod.iloc[:,range(1,IluminaPylumCod.shape[1]-1)].columns.tolist()
for i in range(len(columns)):
    aux = pd.get_dummies(IluminaPylumCod[columns[i]], prefix=(columns[i]))
    aux = aux.loc[:,~aux.columns.str.endswith('_0.0')]
    IluminaPylumCodOH = pd.concat([IluminaPylumCodOH, aux], axis=1)

IluminaPylumCodOH = IluminaPylumCodOH.drop(columns=columns)
fertilCol = IluminaPylumCodOH.pop("Fertil").astype(bool)
IluminaPylumCodOH.insert(IluminaPylumCodOH.shape[1], "Fertil", fertilCol)
IluminaPylumCodOH = IluminaPylumCodOH.set_index('Sample ID')

#IluminaFamilyCodOH
columns = IluminaFamilyCod.iloc[:,range(1,IluminaFamilyCod.shape[1]-1)].columns.tolist()
for i in range(len(columns)):
    aux = pd.get_dummies(IluminaFamilyCod[columns[i]], prefix=(columns[i]))
    aux = aux.loc[:,~aux.columns.str.endswith('_0.0')]
    IluminaFamilyCodOH = pd.concat([IluminaFamilyCodOH, aux], axis=1)

IluminaFamilyCodOH = IluminaFamilyCodOH.drop(columns=columns)
fertilCol = IluminaFamilyCodOH.pop("Fertil").astype(bool)
IluminaFamilyCodOH.insert(IluminaFamilyCodOH.shape[1], "Fertil", fertilCol)
IluminaFamilyCodOH = IluminaFamilyCodOH.set_index('Sample ID')

#IluminaGenusCodOH
columns = IluminaGenusCod.iloc[:,range(1,IluminaGenusCod.shape[1]-1)].columns.tolist()
for i in range(len(columns)):
    aux = pd.get_dummies(IluminaGenusCod[columns[i]], prefix=(columns[i]))
    aux = aux.loc[:,~aux.columns.str.endswith('_0.0')]
    IluminaGenusCodOH = pd.concat([IluminaGenusCodOH, aux], axis=1)

IluminaGenusCodOH = IluminaGenusCodOH.drop(columns=columns)
fertilCol = IluminaGenusCodOH.pop("Fertil").astype(bool)
IluminaGenusCodOH.insert(IluminaGenusCodOH.shape[1], "Fertil", fertilCol)
IluminaGenusCodOH = IluminaGenusCodOH.set_index('Sample ID')

In [28]:
SampleInfo.to_csv(ILUMINA_CLEAN_PATH / "SampleInfo.csv", index=False, encoding='utf-8')
SpermQuality.to_csv(ILUMINA_CLEAN_PATH / "SpermQuality.csv", index=False, encoding='utf-8')

#Ilumina
IluminaPylumCodOH.to_csv(ILUMINA_CLEAN_PATH / "IluminaPylumCodOH.csv", index=True, encoding='utf-8')
IluminaFamilyCodOH.to_csv(ILUMINA_CLEAN_PATH / "IluminaFamilyCodOH.csv", index=True, encoding='utf-8')
IluminaGenusCodOH.to_csv(ILUMINA_CLEAN_PATH / "IluminaGenusCodOH.csv", index=True, encoding='utf-8')

IluminaPylumCod.to_csv(ILUMINA_CLEAN_PATH / "IluminaPylumCod.csv", index=False, encoding='utf-8')
IluminaFamilyCod.to_csv(ILUMINA_CLEAN_PATH / "IluminaFamilyCod.csv", index=False, encoding='utf-8')
IluminaGenusCod.to_csv(ILUMINA_CLEAN_PATH / "IluminaGenusCod.csv", index=False, encoding='utf-8')

IluminaPylum.to_csv(ILUMINA_CLEAN_PATH / "IluminaPylum.csv", index=False, encoding='utf-8')
IluminaFamily.to_csv(ILUMINA_CLEAN_PATH / "IluminaFamily.csv", index=False, encoding='utf-8')
IluminaGenus.to_csv(ILUMINA_CLEAN_PATH / "IluminaGenus.csv", index=False, encoding='utf-8')

#Nanopore
NanoporePylum.to_csv(NANOPORE_CLEAN_PATH / "NanoporePylum.csv", index=False, encoding='utf-8')
NanoporeFamily.to_csv(NANOPORE_CLEAN_PATH / "NanoporeFamily.csv", index=False, encoding='utf-8')
NanoporeGenus.to_csv(NANOPORE_CLEAN_PATH / "NanoporeGenus.csv", index=False, encoding='utf-8')